# Лабораторная работа No5. Сегментация текста

In [1]:
from PIL import Image, ImageDraw, ImageFont
import PIL.ImageOps
import numpy as np
import math
import matplotlib.pyplot as plt

1. Подготовить текст из одной строки в Microsoft Word, пользуясь выбранным
алфавитом и теми же параметрами шрифта. Сделать скриншот и сохранить в
монохромный файл *.bmp, так чтобы вокруг строки не было белого фона.

![title](pictures/Untitled-10.bmp)

In [127]:
alphabeth = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
font = ImageFont.truetype("Times New Roman.ttf", 52)
img = Image.new("L", (len(alphabeth)*50, 50), (0))
draw = ImageDraw.Draw(img)
#w,h=font.getsize(img)
for letter,space in zip(alphabeth,range(0,len(alphabeth))):
    
    shift=12
    if letter == "I" or letter == "J":
        shift=20
    if letter == "W":
        shift=5
    shift=0
    draw.text((0+shift+space*50, 0), letter, (255), font=font)
    
img= to_converted_grayscale(img)
img.save('pictures/artificial.bmp')

2. Реализовать алгоритм расчёта горизонтального и вертикального профиля
изображения.

In [2]:
def to_converted_grayscale(image):
    new_img = image.convert('L')
    new_img = PIL.ImageOps.invert(new_img)
    arr=np.asarray(new_img)
    return new_img
def proj_y(img):
    h=img.size[1]
    arr = np.asarray(img)
    prof=[]
    for y in range(h):
        prof.append(np.sum(arr[y]))
    return prof
def proj_x(img):
    w=img.size[0]
    arr = np.asarray(img).transpose()
    prof=[]
    for x in range(w):
        prof.append(np.sum(arr[x]))
    return prof

3. Реализовать алгоритм сегментации символов в строке на основе профилей с
прореживанием.

In [3]:
def find_zero(arr,trh=255):
    count=0
    for i in arr:
        if i<trh:
            count+=1
    return count

In [4]:
def get_zones_in_row(prof,r):
    w=len(prof)
    x_coords=[]
    flag = False
    x_start = -1
    x_finish = -1
    for i in range(w-r-1):
        count = find_zero(prof[i:i+r])
        if count == 0 and flag == False:
            flag=True
            x_start=i
        if count == r and flag == True:
            flag = False
            x_finish=i-1
            x_coords.append((x_start-1,x_finish+1))
            x_start=-1
            x_finish=-1
    return x_coords

4. Построить профили символов выбранного алфавита.

In [5]:
def get_letters_in_row(prof,img):
    draw = ImageDraw.Draw(img)
    h=img.size[1]
    r = 1
    zones = get_zones_in_row(prof, r)

    letters = [[(i[0],0),(i[1],0),(i[0],h-1),(i[1],h-1)] for i in zones]
    for x in zones:
        draw.line((x[0], 0, x[0], h-1), fill=128,width=1)
        draw.line((x[1], 0, x[1], h - 1), fill=128,width=1)
    img.show()
    return letters
def letters_prof(img,letters):
    w=img.size[0]
    h = img.size[1]
    k=0
    s=list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
    #s=list("JIVEDFOXNYMPHGRABSQUICKWALTZ")
    for i in letters:
        letter_img = img.crop((i[0][0], i[0][1], i[3][0], i[3][1]))
        w = letter_img.size[0]
        h = letter_img.size[1]
        prof_x = proj_x(letter_img)
        x=[j for j in range(w)]
        prof_y=proj_y(letter_img)
        y = [j for j in range(h)]
        graph_x = plt.plot(x, prof_x)
        plt.savefig(fname="pictures/profiles/" + s[k] + "_x", fmt='png')
        plt.clf()
        graph_y = plt.plot(y, prof_y)
        plt.savefig(fname="pictures/profiles/"+s[k]+"_y", fmt='png')
        plt.clf()
        letter_img=PIL.ImageOps.invert(letter_img)
        letter_img.save('pictures/alphabeth/' + s[k] + '.bmp')
        k+=1
    return

In [6]:
def main():
    path_to_img="pictures/Untitled-10.bmp"
    image = Image.open(path_to_img)
    new_img=to_converted_grayscale(image)
    cropy_img=new_img.crop((0,0,image.size[0],49))
    #cropy_img.show()
    proj_1=proj_y(new_img)
    #print(proj_1)
    proj_2 = proj_x(cropy_img)
    #print(proj_2)
    letters=get_letters_in_row(proj_2,image)
    print(letters)
    #print(len(letters))
    letters_prof(new_img,letters)

В результате работы алгоритма возвращается массив с
координатами обрамляющих символы прямоугольников, упорядоченные в порядке
чтения слева направо, сверху вниз.

In [7]:
%%time
main()

[[(0, 0), (27, 0), (0, 51), (27, 51)], [(32, 0), (53, 0), (32, 51), (53, 51)], [(57, 0), (81, 0), (57, 51), (81, 51)], [(86, 0), (111, 0), (86, 51), (111, 51)], [(116, 0), (135, 0), (116, 51), (135, 51)], [(141, 0), (158, 0), (141, 51), (158, 51)], [(162, 0), (188, 0), (162, 51), (188, 51)], [(195, 0), (219, 0), (195, 51), (219, 51)], [(226, 0), (231, 0), (226, 51), (231, 51)], [(235, 0), (247, 0), (235, 51), (247, 51)], [(255, 0), (275, 0), (255, 51), (275, 51)], [(280, 0), (297, 0), (280, 51), (297, 51)], [(301, 0), (336, 0), (301, 51), (336, 51)], [(344, 0), (368, 0), (344, 51), (368, 51)], [(374, 0), (403, 0), (374, 51), (403, 51)], [(409, 0), (429, 0), (409, 51), (429, 51)], [(433, 0), (466, 0), (433, 51), (466, 51)], [(468, 0), (489, 0), (468, 51), (489, 51)], [(492, 0), (512, 0), (492, 51), (512, 51)], [(513, 0), (538, 0), (513, 51), (538, 51)], [(541, 0), (566, 0), (541, 51), (566, 51)], [(570, 0), (597, 0), (570, 51), (597, 51)], [(599, 0), (640, 0), (599, 51), (640, 51)], [(6

<Figure size 432x288 with 0 Axes>